In [1]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown
import numpy.ma as ma
import scipy as sp


In [2]:
class OneToOneITU():
    def __init__(self, n, m, parameters = (None,None) , lbs=(0, 0)):
        self.n = n
        self.m = m
        self.lb_U, self.lb_V = lbs
     
       
        self.A_ij = parameters[0]
        self.B_ij = parameters[1]
        
        self.max_A_j = np.max(parameters[0],0)

    def D_ij(self, U_i, V_j):
            return (U_i[:, None] + self.A_ij * V_j[None, :] - self.B_ij)/(1+ self.A_ij)

    def get_U_ij(self, V_j, i_idx ,j_idx = None):
        if j_idx is None:
            return self.B_ij[i_idx] - self.A_ij[i_idx] * V_j[None, :]
        else:
            return self.B_ij[i_idx,j_idx] - self.A_ij[i_idx,j_idx] * V_j[None, :]

    def get_V_ij(self, U_i, j_idx ,i_idx = None):
        if i_idx is None:
            return (self.B_ij[:,j_idx] -  U_i[:,None])/ self.A_ij[:,j_idx]
        else:
            return (self.B_ij[i_idx,j_idx] -  U_i[:,None])/ self.A_ij[i_idx,j_idx]

    def U_ij(self, i,j, V_j):
            return self.B_ij[i,j] - self.A_ij[i,j] * V_j

    def V_ij(self,i,j,U_i):
            return (self.B_ij[i,j] -  U_i)/ self.A_ij[i,j]

In [3]:
def generate_example(self, var_i, var_j, random_seed):
    np.random.seed(random_seed)
    k = 2

    θ_true = np.array([-0.1,0.1, 1,2])

    X_i = np.random.normal([5,var_i],[5,var_i], size = [self.n, k]).round(0)
    Y_j = np.random.normal([5,var_j],[5,var_j], size = [self.m, k]).round(0)
    #X_ijk = np.random.choice(10, size = (self.n_i,m_j, k))

    A = np.exp( np.abs((X_i[:,None,:]- Y_j[None,:,:]))@  θ_true[:k]).round(1)+.001
    B = np.abs((X_i[:,None,:]*Y_j[None,:,:])) @ θ_true[k:2*k] 
    print(len(np.unique(X_i)))
    self.A_ij = A
    self.B_ij = B
    self.max_A_j = np.max(A, axis= 0)

OneToOneITU.generate_example = generate_example

In [4]:
def check_primal_feas(self, mu_ij):
    print(f"i: { np.all(np.sum(mu_ij,axis=1) <= 1)}, j:  {np.all(np.sum(mu_ij,axis=0) <= 1) }")
    print(f"#matched: {int(np.sum(mu_ij))} over {np.minimum(self.n,self.m)}")

OneToOneITU.check_primal_feas = check_primal_feas

def check_IR(self,mu_ij,U_i,V_j, output = False):
    IR_i = np.sum((U_i - self.lb_U)* (1- np.sum(mu_ij,axis=1)))
    IR_j = np.sum((V_j - self.lb_V) * (1 - np.sum(mu_ij,axis=0))) 
    print(f"i: { IR_i}, j:  {IR_j }")
    if output is True:
        IR_i = (U_i - self.lb_U)* (np.sum(mu_ij,axis=1) - 1) >= 0 
        IR_j = (V_j - self.lb_V) * (np.sum(mu_ij,axis=0) - 1) >= 0
        return IR_i, IR_j
    
OneToOneITU.check_IR = check_IR

def check_CS(self,U_i,V_j, output = None):
    CS = np.minimum(self.D_ij(U_i,V_j),0)
    if output:
        return CS, np.where(CS < 0 )
    print(f"min_ij D_ij : {(self.D_ij(U_i,V_j)).min()} ")
    
OneToOneITU.check_CS = check_CS 

def check_all(self,eq):
    mu_ij, U_i, V_j = eq 
    print("FEAS")
    self.check_primal_feas(mu_ij)
    print("________________")
    print("ε-CS")
    self.check_CS(U_i, V_j )
    print("________________")
    print("IR")
    self.check_IR(mu_ij, U_i, V_j )

OneToOneITU.check_all = check_all

def check_CS_i_or_j(self,u_i, v_j, side = 0):
    if side == 0:
        return u_i[:,None] - self.get_U_ij(v_j, np.arange(self.n))
    else:
        return v_j[None,:] - self.get_V_ij(u_i, np.arange(self.m))
    
OneToOneITU.check_CS_i_or_j = check_CS_i_or_j

# Auction Algorithm

In [5]:
def forward_auction_serial(self, data, tol_ε = None):
    mu_i , V_j = data.copy()
    while np.any(mu_i == self.m +1):
        for i in np.where(mu_i== self.m +1)[0]:
            U_ij = self.U_ij(i,np.arange(self.m),V_j)
            j_star = np.argmax(U_ij)
            if U_ij[j_star] <= self.lb_U:
                mu_i[i] = self.m
            else:
                w_i = np.maximum(np.max(np.where( np.arange(self.m) == j_star, -np.inf, U_ij)), self.lb_U)
                V_j[j_star] = self.V_ij(i,j_star, w_i - tol_ε)
                mu_i[mu_i == j_star] = self.m+1
                mu_i[i] = j_star
                
    U_i = np.ones(self.n) * self.lb_U
    matched_i = mu_i < self.m
    U_i[matched_i] = self.get_U_ij(V_j, matched_i)[np.arange(matched_i.sum()), mu_i[matched_i]]
    return mu_i, U_i, V_j
    

OneToOneITU.forward_auction_serial = forward_auction_serial   

### Backward


In [6]:
def reverse_auction_serial(self, data = None, tol_ε = None):
    mu_j , U_i = data.copy()
    iter= 0 
    while np.any(mu_j == self.n +1) and iter < 1000000:
        iter += 1
        for j in np.where(mu_j== self.n +1)[0]:
            V_ij = self.V_ij(np.arange(self.n),j,U_i)
            i_star = np.argmax(V_ij)
            if V_ij[i_star] <= self.lb_V:
                mu_j[j] = self.n
            else:
                β_j = np.maximum(np.max(np.where( np.arange(self.n) == i_star, -np.inf, V_ij)), self.lb_V)
                U_i[i_star] = self.U_ij(i_star,j, β_j - tol_ε)
                mu_j[mu_j == i_star] = self.n+1
                mu_j[j] = i_star
                
                
    V_j = np.ones(self.m) * self.lb_V
    matched_j = mu_j < self.n 
    V_j[matched_j] = self.get_V_ij(U_i, matched_j )[mu_j[matched_j],np.arange(matched_j.sum()) ]

    return mu_j, U_i, V_j
    

OneToOneITU.reverse_auction_serial = reverse_auction_serial


In [7]:
def forward_backward_scaling_serial(self, tol_ε ,tol_ε_obj, scaling_factor):
    v_j = np.ones(self.m) * self.lb_V
    mu_i = np.ones(self.n, dtype= int) * (self.m+1)
    
    print("_________")
    while tol_ε > tol_ε_obj:


        mu_i_iter, u_i,v_j = self.forward_auction_serial([mu_i, v_j], tol_ε= tol_ε)

        tol_ε *=  scaling_factor
        mu_j = np.ones(self.m, dtype= int) * (self.n+1)
        mu_j_iter , u_i,v_j = self.reverse_auction_serial([mu_j, u_i], tol_ε= tol_ε)
        #self.check_IR(mu_ij_01, u_i,v_j)
        
        #self.check_CS(u_i,v_j)
        #self.check_IR(mu_ij_01, u_i,v_j)
        print("_________")   
        # if np.all(self.check_CS(u_i,v_j,True)[0]>= - tol_ε_obj):
        #     return mu_ij_01, u_i, v_j
    
    # mu_ij_01, u_i, v_j = self.drop_for_scaling(mu_ij_01, u_i, v_j , tol_ε)
    
    mu_ij_01 = mu_i_iter[:,None] == np.arange(self.m)[None,:]
    return mu_ij_01, u_i, v_j
OneToOneITU.forward_backward_scaling_serial =forward_backward_scaling_serial  

In [15]:
n_i =  500
m_j  = 600
np.random.seed(123)
A_ =  np.ones([n_i,m_j])#np.random.choice([.25,.5,1,2,4], size= [n_i,1] ) * np.random.choice([.25,.5,1,2,4], size= [1, m_j] ) #  #np.random.choice([.25,.5,1,2,4], size= [n_i,m_j])
B_ = (np.random.randint(1,3, size= [ n_i,1]) * np.random.randint(1,3, size= [ 1,m_j]))**2 +1 #np.random.choice([200, 300], size=(n_i, m_j), p=[.9,.1 ]) * 100#np.random.randint(n_i/2,n_i,size = [n_i,m_j]) # np.random.choice([0,1], size= [n_i,m_j])*10000
example_mkt_2 = OneToOneITU(  n_i,m_j,parameters=(A_,B_))

In [17]:
mu_ij_01, u_i,v_j = example_mkt_2.forward_backward_scaling_serial(1,.0001, 1/4)

_________
_________
_________
_________
_________
_________
_________
_________


In [11]:
example_mkt_2.check_all((mu_ij_01, u_i,v_j))

FEAS
i: True, j:  True
#matched: 50 over 50
________________
ε-CS
min_ij D_ij : -4.999999999810711e-06 
________________
IR
i: 0.0, j:  0.0
